In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error



In [5]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlopsproject1/mlruns/1', creation_time=1717182686025, experiment_id='1', last_update_time=1717182686025, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [6]:
feb_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
jan_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [7]:
df_columns = jan_df.shape[1]
df_columns

19

In [8]:
jan_df['duration'] = (jan_df["tpep_dropoff_datetime"] - jan_df["tpep_pickup_datetime"]).dt.total_seconds() / 60
jan_sd = jan_df['duration'].std()
# df = jan_df[(Jjn_df.duration >= 1) & (jan_df.duration <= 60)]
jan_filtered = jan_df[(jan_df.duration >= 1) & (jan_df.duration <= 60)]
# categorical = ['PULocationID', 'DOLocationID']
# numerical = ['trip_distance']
jan_sd
# df[categorical] = df[categorical].astype(str)

42.594351241920904

In [9]:
jan_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,13.983333
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,19.450000
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,24.516667
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,13.000000


In [10]:
fraction_remaining = len(jan_filtered) / len(jan_df)
fraction_remaining

0.9812202822125979

In [11]:
jan_filtered['PULocationID'] = jan_filtered['PULocationID'].astype(str)
jan_filtered['DOLocationID'] = jan_filtered['DOLocationID'].astype(str)
train_dicts = jan_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(train_dicts)

/tmp/ipykernel_53767/1445089871.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_filtered['PULocationID'] = jan_filtered['PULocationID'].astype(str)
/tmp/ipykernel_53767/1445089871.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_filtered['DOLocationID'] = jan_filtered['DOLocationID'].astype(str)


In [12]:
# dv = DictVectorizer(sparse=True)

In [13]:
num_columns = X_train.shape[1]

In [14]:
num_columns

515

In [15]:
y_train = jan_filtered['duration'].values

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [16]:
train_predictions = model.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(y_train, train_predictions))

In [17]:
rmse_train

7.649261932106969

In [18]:
print("executed6")
feb_df['duration'] = feb_df['duration'] = (feb_df["tpep_dropoff_datetime"] - feb_df["tpep_pickup_datetime"]).dt.total_seconds() / 60
print("executed7")
# feb_df.duration = feb_df.duration.apply(lambda td: td.total_seconds() / 60)

print("executed1")
feb_df['PULocationID'] = feb_df['PULocationID'].astype(str)
print("executed2")
feb_df['DOLocationID'] = feb_df['DOLocationID'].astype(str)
print("executed3")
feb_data_dicts = feb_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')
print("executed4")
feb_df['PULocationID'] = feb_df['PULocationID'].astype(str)
feb_df['DOLocationID'] = feb_df['DOLocationID'].astype(str)
feb_data_dicts = feb_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Transform February data using the dictionary vectorizer
X_val = dv.fit_transform(feb_data_dicts)

# Calculate the RMSE on the validation data
y_val = feb_df['duration'].values
val_predictions = model.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, val_predictions))

# Print results
print("Number of columns in the feature matrix:", num_columns)
print("RMSE on training data:", rmse_train)
print("RMSE on validation data:", rmse_val)

executed6
executed7
executed1
executed2
executed3
executed4


: 

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, model), f_out)

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("developer","logesh")
    
    mlflow.log_param("train-data-path", "data/yellow_tripdata_2023-01.parquet")
    mlflow.log_param("val-data-path", "data/yellow_tripdata_2023-02.parquet")
    
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

/home/codespace/anaconda3/envs/projectenv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
